Setup imports

In [135]:
import folium
import geopandas as gpd
import pandas as pd

Setup Folium map object

In [136]:
m = folium.Map()
folium.LayerControl().add_to(m)

Read in Local Authority data, and convert from BNG to WGS84 to be compatible with Folium/Leaflet

In [137]:
la_url = "http://lle.gov.wales/catalogue/item/HighWaterMark.json"
la_geom_df = gpd.read_file(la_url)
la_geom_df.crs = {"init": "epsg:27700"}
la_geom_df = la_geom_df.to_crs({"init": "epsg:4326"})
la_geom_df.head()

,id,ogc_fid,name_en,name_cy,code,geometry
0,localauthorities.1,1,Swansea,Abertawe,W06000011,"MULTIPOLYGON (((-4.06231 51.69310, -4.06231 51..."
1,localauthorities.2,2,Blaenau Gwent,Blaenau Gwent,W06000019,"MULTIPOLYGON (((-3.16724 51.70990, -3.16787 51..."
2,localauthorities.3,3,Vale of Glamorgan,Bro Morgannwg,W06000014,"MULTIPOLYGON (((-3.53284 51.48161, -3.53272 51..."
3,localauthorities.4,4,Newport,Casnewydd,W06000022,"MULTIPOLYGON (((-3.08265 51.50182, -3.08343 51..."
4,localauthorities.5,5,Conwy,Conwy,W06000003,"MULTIPOLYGON (((-3.51750 53.28546, -3.51744 53..."


Read in Welsh language data
Column is renamed to join with Local Authority dataset

In [138]:
welsh_lang_df = pd.read_csv("welsh_lang_data.csv")
welsh_lang_df.rename(columns={'Unnamed: 0': 'name_en'}, inplace=True)
welsh_lang_df['name_en'].str.rstrip()
welsh_lang_df.head()

,name_en,All aged 3 or over (1),"Yes, can speak Welsh","No, cannot speak Welsh",Percentage of people who say they can speak Welsh (2)
0,Wales,3034400,884300,2147800,29.163972
1,Isle of Anglesey,68100,45100,22900,66.275410
2,Gwynedd,118800,90700,28000,76.386139
3,Conwy,111800,41900,69900,37.473720
4,Denbighshire,91200,31200,59800,34.278713


Create choropleth layer and add to map

In [140]:
choropleth = folium.Choropleth(
    geo_data=la_geom_df,
    name="choropleth",
    data=welsh_lang_df,
    columns=["name_en", "Percentage of people who say they can speak Welsh (2)"],
    key_on="feature.name_en",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Percentage of people who say they can speak Welsh (%)",
)
choropleth.add_to(m)

Set map bounds, and render in Juputer Notebook

In [141]:
m.fit_bounds(choropleth.get_bounds())
m

Finally, save our Folium map to html for deployment

In [142]:
output_file = "index.html"
m.save(output_file)